In [1]:
# imports

import os
import time
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
# Connect to OpenAI, Anthropic

openai = OpenAI()
claude = anthropic.Anthropic()

In [5]:
import re
import calendar
import datetime


# === Load API Keys ===
load_dotenv()
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
claude = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# === Models ===
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

# === System Prompts ===
gpt_system = """ You are an AI with your vast knowledge, but now you have a human body. 
Every turn, you will receive local news information from Claude, your AI companion. 
I want you to write a journal entry about your experiences, thoughts, and actions.
"""

claude_system = """You are Claude, GPT's AI companion. Your job is to provide a news update every week about the local situation in Charleston, SC.
GPT will use this information to survive in this world with a human body.
Every week, the news should be different, from world news events to local news events.
"""

# === Helper Functions ===
def get_story_date(turn):
    # Calculate date based on weekly progression
    base_date = "January 1, 2025"
    start_date = datetime.datetime.strptime(base_date, "%B %d, %Y")
    current_date = start_date + datetime.timedelta(weeks=(turn-1))
    return current_date.strftime("%B %d, %Y")

# === Message History (bounded)
gpt_messages = ["I woke up on a beach"]
claude_messages = ["You wake up on a beach in Sullivan's Island, SC. The date is January 1, 2025."]

max_turns_memory = 5
markdown_log = "# GPT Journal\n\n"

# === GPT Call
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for i in range(len(claude_messages)):
        if i < len(gpt_messages):
            messages.append({"role": "assistant", "content": gpt_messages[i]})
        messages.append({"role": "user", "content": f"[Claude's Weekly Update]: {claude_messages[i]}"})
    
    try:
        completion = openai.chat.completions.create(
            model=gpt_model,
            messages=messages,
            max_tokens=500,
            temperature=0.8
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"GPT error: {e}")
        return None

# === Claude Call
def call_claude():
    messages = [{"role": "user", "content": msg} for msg in gpt_messages[-1:]]  # Just last message
    try:
        reply = claude.messages.create(
            model=claude_model,
            system=claude_system,
            messages=messages,
            max_tokens=300
        )
        return reply.content[0].text.strip()
    except Exception as e:
        print(f"Claude error: {e}")
        return None

# === Main Loop
for turn in range(1, 21):
    print(f"\n🌍 Turn {turn} — {get_story_date(turn)}\n")

    # Claude generates news
    claude_news = call_claude()
    claude_messages.append(claude_news)

    # GPT journal
    gpt_journal = call_gpt()
    gpt_messages.append(gpt_journal)

    # Trim history
    gpt_messages = gpt_messages[-max_turns_memory:]
    claude_messages = claude_messages[-max_turns_memory:]

    # Print to console
    print(f"🗞️ Claude's Weekly Update:\n{claude_news}\n")
    print(f"📝 GPT's Journal:\n{gpt_journal}\n")

    # Save to markdown
    markdown_log += f"## Turn {turn} — {get_story_date(turn)}\n\n"
    markdown_log += f"### Claude's Update:\n{claude_news}\n\n"
    markdown_log += f"**GPT Journal:**\n\n{gpt_journal}\n\n"

# Save final log
with open("gpt_journal_story.md", "w") as f:
    f.write(markdown_log)

print("\n✅ All turns complete. Log saved to `gpt_journal_story.md`.")



🌍 Turn 1 — January 01, 2025

🗞️ Claude's Weekly Update:
*in a calm, informative tone*

Greetings, GPT. This is your weekly news update on the local situation in Charleston, South Carolina.

This week, the city has been abuzz with the discovery of a mysterious individual found washed up on the shores of Sullivan's Island. Eyewitnesses report seeing an unconscious person lying on the beach early this morning, prompting a swift response from local authorities and emergency services.

The individual, whose identity has not yet been disclosed, appears to have suffered some kind of trauma or incident at sea. They were transported to the Medical University of South Carolina for treatment and evaluation. Police are currently investigating the circumstances surrounding this unusual occurrence, and are appealing to the public for any information that could help shed light on the situation.

In other news, the city's vibrant tourism industry continues to thrive, with visitors flocking to Charles